In [2]:
def RA():
    ## Importing Required packeges
    import pandas as pd
    import numpy as np
    from datetime import date, datetime, timedelta
    import time
    import datacompy
    import os
    from pyspark.sql import Row
    from selenium import webdriver
    from selenium.webdriver.common.by import By 
    from selenium.webdriver.support.ui import WebDriverWait 
    import warnings
    warnings.filterwarnings("ignore")
    
    #### Open browser
    browser = webdriver.Chrome(executable_path='C:\\Users\\11201312\\Downloads\\chromedriver_win32\\chromedriver.exe')
    browser.get("https://clinicaltrials.gov/ct2/results?cond=Rheumatoid+Arthritis&term=&cntry=&state=&city=&dist=")
    
    ### Dowloading data from browser
    browser.find_element_by_xpath('/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[1]/div[3]/a').click()
    browser.find_element_by_xpath('/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[1]/div[4]/div/div/div/form[1]/div[1]/div[2]/select/option[4]').click()
    browser.find_element_by_xpath('/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[1]/div[4]/div/div/div/form[1]/div[3]/div[2]/select/option[2]').click()
    browser.find_element_by_xpath('/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[1]/div[4]/div/div/div/form[1]/div[4]/div[2]/select/option[4]').click()
    browser.find_element_by_xpath('/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[1]/div[4]/div/div/div/form[1]/div[5]/input').click()                               
    
    time.sleep(20)
    browser.close()
    ## Read data from downloads
    df = pd.read_csv('C:/Users/11201312/Downloads/SearchResults.csv')
    print("Shape of downloaded data: ", df.shape)
    
    df.head()
    
    ## Import NCT ids alog with its phase shared by KC team
    kc = pd.read_excel('C:/Users/11201312/Documents/Techwave/GSK_Pilot_Project03032020/CT_RA_Tracker/RA__KC_Trials.xlsx')
    print("Shape of NCT, Phase data Share by KC: ", kc.shape)
    
    ######################################################################
    #### Reading TrialTrove full data #####
    ttdata = pd.read_csv("C:\\Users\\11201312\\Desktop\\output1\\trial\\trial\\trial"+datetime.now().strftime("%Y%m%d")+".csv")
    print("Shape of vlookup "+str(ttdata.shape))
    #ttdata.head()
    
    ################ Read Trial PatientSegment Data #####################
    ttpatientsegment = pd.read_csv("C:\\Users\\11201312\\Desktop\\output1\\trial\\Clinical\\trialPatientSegments.csv")
    print("Shape of trialPatientSegments ", ttpatientsegment.shape)
    
    ## Grouping by trialid
    ttpatientsegment1 = ttpatientsegment.groupby('trialId', as_index=False).agg(lambda x: x.tolist())
    
    ## Coverting list columns to string
    ttpatientsegment1["trialPatientSegments1"] = ttpatientsegment1['trialPatientSegments'].astype(str).str.replace('\[|\]|\'', '') 
    print("Shape of trialPatientSegments1 ", ttpatientsegment1.shape)

    ## Export grouped/modified data to the location
    #ttpatientsegment1.to_csv("C:\\Users\\11201116\\Documents\\Techwave\\GSK_Pilot_Project\\output1\\Clinical\\trialPatientSegments1.csv")
    
    ### Apply "Vlookup" in padas for trialTrove data and PatientSegment1 data
    tpdata = pd.merge(ttdata, ttpatientsegment1[["trialId", "trialPatientSegments1"]], on='trialId')#.drop_duplicates(["trialId"])
    print("Vlookup TT, PatientSegment data: " + str(tpdata.shape))
    #tdata.head()

    ################# Read trialtrialPrimaryDrugsTested data #########################
    drugdata = pd.read_csv("C:\\Users\\11201312\\Desktop\\output1\\trial\\Clinical\\trialtrialPrimaryDrugsTested.csv")
    print("Shape of drugdata: ", drugdata.shape)
    
    ## Grouping by trialid
    drugdata1 = drugdata.groupby('trialId', as_index=False).agg(lambda x: x.tolist())
        
    ## Coverting list columns to string
    drugdata1["drugName1"] = drugdata1['drugName'].astype(str).str.replace('\[|\]|\'', '')
    drugdata1["drugPrimaryName1"] = drugdata1['drugPrimaryName'].astype(str).str.replace('\[|\]|\'', '')

    print("Shape of drugdata1: ",drugdata1.shape)

    ## Export grouped/modified data to the location
    #drugdata1.to_csv("C:\\Users\\11201116\\Documents\\Techwave\\GSK_Pilot_Project\\output1\\Clinical\\trialtrialPrimaryDrugsTested1.csv")

    ### Apply "Vlookup" in padas for tpdata and drugdata1
    tpddata = pd.merge(tpdata, drugdata1[["trialId", "drugName1"]], on='trialId')#.drop_duplicates(["trialId"])
    print("Vlookup of TT PatientSegment & Drug data shape: " + str(tpddata.shape))
    #tdata.head()

    ### Keep Required columns and store it in a dataframe
    tt_ct = tpddata[['trialId', 'trialCountries', 'ctGovListedLocations', 'trialObjective', 'trialInclusionCriteria', 'trialExclusionCriteria', 
               'trialTreatmentPlan', 'trialProtocolIDs', 'trialSupportingUrls', 'trialTags', 'trialOutcomes', 'trialTargetAccrual', 
               'trialPatientSegments1', 'trialPrimaryEndpointsReported','drugName1']]
    print("tt_ct shape: ", tt_ct.shape)
    
    ##############################################################
    
      
    ### Integrate TrialTrove code to get required fields to add it to CT data ###
    ###### Apply this Vlookup to final aggrigated TT data ######
    
    ### Apply this Vlookup to final aggrigated TT data
    tt1 = tt_ct.merge(kc, on='trialId', how='right')
    print("Shape of vlookup dataframe: "+ str(tt1.shape))
    tt1.head()
    
    # apply "Vlookup" in padas to Merge/Vlookup TT data to CT data
    Updated1 = df.merge(tt1, on='NCT Number')
    print(Updated1.shape)
    Updated1.head()
    
    ### Create a DataFrame to get required data fields ###
    Data = pd.DataFrame()
    
    Data['Clinical Trial Identifier'] = Updated1['NCT Number']
    Data['Trial Title'] = Updated1['Title']
    Data['Title Acronym'] = Updated1['Acronym']
    Data['Competitor Asset'] = Updated1['Interventions']
    Data['Condition/Disease'] = Updated1['Conditions']
    Data['Trial Phase'] = Updated1['trialPhase']
    Data['Trial Status'] = Updated1['Status']
    Data['Sponsor(s)'] = Updated1['Sponsor/Collaborators']
    Data['Target Accrual'] = Updated1['trialTargetAccrual']
    Data['Actual Accrual'] = Updated1['Enrollment']
    Data['Countries'] = Updated1['trialCountries']
    Data['No.of Countries'] = Updated1.trialCountries.str.count(',')+1  # Count no.of countries by counting comma, b/s countries are seperated with comma
    Data['Geographical Reach'] = np.where(Data['No.of Countries'] == 1, Data.Countries, 
                                          np.where(Data['No.of Countries'] > 1, 'Global', 'Null'))  # Country/Global categorization
    Data['Study Start Date'] = Updated1['Start Date']
    Data['Primary Completion Date'] = Updated1['Primary Completion Date']
    Data['Study Completion Date'] = Updated1['Completion Date']
    Data['Trial Objective(s)'] = Updated1['trialObjective']
    Data['Outcome Measures'] = Updated1['Outcome Measures']
    Data['Primary Endpoints Reported Date'] = Updated1['trialPrimaryEndpointsReported']
    #Data['Other EndPoint'] = Updated1['Outcome Measures']
    Data['Inclusion Criteria'] = Updated1['trialInclusionCriteria']
    Data['Exclusion Criteria'] = Updated1['trialExclusionCriteria']
    Data['Study Design'] = Updated1['Study Designs']
    Data['Patient Segments'] = Updated1['trialPatientSegments1']
    Data['Treatment Plan'] = Updated1['trialTreatmentPlan']
    Data['Dosing/Comparator'] = Updated1['trialTreatmentPlan']
    Data['Age'] = Updated1['Age']
    Data['Gender'] = Updated1['Gender']
    Data['ROA'] = Updated1['drugName1']
    Data['Trial Tag/Attribute'] = Updated1['trialTags']
    Data['Trial Outcome(s)'] = Updated1['trialOutcomes']
    Data['Other Trial Identifier(s)'] = Updated1['trialProtocolIDs']
    Data['Additional Source'] = Updated1['trialSupportingUrls']
    Data['Last CT.gov Update noted on'] = Updated1['Last Update Posted']
    Data['TrialtroveId'] = Updated1['trialId']
    Data['Date'] = str(date.today())
    
    ### Saving today updated data in local
    Data.to_excel('C:/Users/11201312/Documents/Techwave/GSK_Pilot_Project03032020/CT_RA_Tracker/ct_tt_ra_Updated'+str(date.today())+'.xlsx', index=False)
    
    ### Sape of Latest file
    print("Shape of latest CT dataframe: ", Data.shape)
    
    #### Print latest file columns
    Data.columns
    
    ### Subset latest updated data for comparision based on preority 
    data_comp = Data[['Clinical Trial Identifier', 'Competitor Asset', 'Sponsor(s)', 'Trial Title', 'Condition/Disease', 
                  'Trial Phase', 'Trial Status', 'Other Trial Identifier(s)', 'Target Accrual', 'Actual Accrual', 
                  'Title Acronym', 'Countries', 'Trial Objective(s)', 'Study Design', 'Patient Segments', 'Treatment Plan', 
                  'Dosing/Comparator', 'ROA', 'Outcome Measures', 'Study Start Date', 'Primary Completion Date', 
                  'Study Completion Date', 'Inclusion Criteria', 'Exclusion Criteria', 'Trial Tag/Attribute', 
                  'Other Trial Identifier(s)', 'Trial Outcome(s)']]
    
    ## Shape of comparision data
    print("Shape of Comparision dataframe: "+str(data_comp.shape))
    
    ### Exporting Comparision sheet to local repository 
    data_comp.to_excel('C:/Users/11201312/Documents/Techwave/GSK_Pilot_Project03032020/CT_RA_Tracker/ct_tt_ra'+str(date.today())+'.xlsx', index=False)
                        
    #Remove Search resulta data from downloads. Run this line of code after completing regular comparision task.
    import os
    os.remove('C:/Users/11201312/Downloads/SearchResults.csv')
    
    ########################################################################################################################
    ##### CT Data to display ##### 
    Data['Registry'] = "ClinicalTrialgov"
    Data['IDs'] = Data['Clinical Trial Identifier']
    
    Data_Display_CT = Data[['Clinical Trial Identifier', 'Registry', 'IDs', 'Trial Title', 'Title Acronym', 'Competitor Asset', 
                            'Condition/Disease', 'Trial Phase', 'Trial Status', 'Sponsor(s)', 'Target Accrual', 'Actual Accrual',
                            'Countries', 'Study Start Date', 'Primary Completion Date', 'Study Completion Date', 
                            'Trial Objective(s)', 'Outcome Measures', 'Primary Endpoints Reported Date', 'Inclusion Criteria', 
                            'Exclusion Criteria', 'Study Design', 'Patient Segments', 'Treatment Plan', 'Dosing/Comparator', 
                            'Age', 'Gender', 'ROA', 'Trial Tag/Attribute', 'Trial Outcome(s)', 'Other Trial Identifier(s)', 
                            'Additional Source', 'Last CT.gov Update noted on']]

    print("CT Data to display: ", Data_Display_CT.shape)
    
    ##### TT Data to dispaly #####
    ### Apply "Vlookup" in padas for tpddata and Updated1
    tpddata1 = pd.merge(tpddata, Updated1[["trialId", "NCT Number"]], on='trialId')
    print("Vlookup tpddata and Updated1: " + str(tpddata1.shape))
    #tdata.head()
    
    ## Dataframe for TT data ##
    Data_Display_TT = pd.DataFrame() 
    
    Data_Display_TT['Clinical Trial Identifier'] = tpddata1['NCT Number']
    Data_Display_TT['Registry'] = 'Trialtrove'
    Data_Display_TT['IDs'] = tpddata1['trialId']
    Data_Display_TT['Trial Title'] = tpddata1['trialTitle']
    Data_Display_TT['Title Acronym'] = Data_Display_CT['Title Acronym']
    Data_Display_TT['Competitor Asset'] = tpddata1['drugName1']
    Data_Display_TT['Condition/Disease'] = Data_Display_CT['Condition/Disease']
    Data_Display_TT['Trial Phase'] = tpddata1['trialPhase']
    Data_Display_TT['Trial Status'] = tpddata1['trialStatus']
    Data_Display_TT['Sponsor(s)'] = tpddata1['trialSponsors']
    Data_Display_TT['Target Accrual'] = tpddata1['trialTargetAccrual']
    Data_Display_TT['Actual Accrual'] = tpddata1['trialActualAccrual']
    Data_Display_TT['Countries'] = tpddata1['trialCountries']
    Data_Display_TT['Study Start Date'] = tpddata1['trialStartDate']
    Data_Display_TT['Primary Completion Date'] = tpddata1['trialPrimaryCompletionDate']
    Data_Display_TT['Study Completion Date'] = tpddata1['trialPrimaryCompletionDate']
    Data_Display_TT['Trial Objective(s)'] = tpddata1['trialObjective']
    Data_Display_TT['Outcome Measures'] = tpddata1['TRIALPRIMARYENDPOINTdetails']
    Data_Display_TT['Primary Endpoints Reported Date'] = tpddata1['trialPrimaryEndpointsReported']
    Data_Display_TT['Inclusion Criteria'] = tpddata1['trialInclusionCriteria']
    Data_Display_TT['Exclusion Criteria'] = tpddata1['trialExclusionCriteria']
    Data_Display_TT['Study Design'] = tpddata1['trialStudyDesign']
    Data_Display_TT['Patient Segments'] = tpddata1['trialPatientSegments1']
    Data_Display_TT['Treatment Plan'] = tpddata1['trialTreatmentPlan']
    Data_Display_TT['Dosing/Comparator'] = tpddata1['trialTreatmentPlan']
    Data_Display_TT['Age'] = tpddata1['TRIALPATIENTCRITERIAminAge']
    Data_Display_TT['Gender'] = tpddata1['TRIALPATIENTCRITERIAgender']
    Data_Display_TT['ROA'] = tpddata1['drugName1']
    Data_Display_TT['Trial Tag/Attribute'] = tpddata1['trialTags']
    Data_Display_TT['Trial Outcome(s)'] = tpddata1['trialOutcomes']
    Data_Display_TT['Other Trial Identifier(s)'] = tpddata1['trialProtocolIDs']
    Data_Display_TT['Additional Source'] = tpddata1['trialSupportingUrls']
    Data_Display_TT['Last CT.gov Update noted on'] = tpddata1['trialLastModifiedDate']

    print("TT data to dispaly: ", Data_Display_TT.shape)
    
    ########################### Appending CT and TT data to display it in CI dashboard ###########################
    frames = [Data_Display_CT, Data_Display_TT]  ### Createing a frame to append CT & TT trials 
    CT_TT_Display = pd.concat(frames)  ### Concatenate all dataframes in frames
    print("History df shape: ", CT_TT_Display.shape)
    
    CT_TT_Display.to_excel('C:/Users/11201312/Documents/Techwave/GSK_Pilot_Project03032020/CT_RA_Tracker/ct_tt_ra_display.xlsx', index=False)
        
    ######################################################################################################################## 
    ######################################## Comparision of two dataframes #################################################
    ########################################################################################################################
    df1 = pd.read_excel("C:/Users/11201312/Documents/Techwave/GSK_Pilot_Project03032020/CT_RA_Tracker/ct_tt_ra.xlsx")
    df2 = pd.read_excel("C:/Users/11201312/Documents/Techwave/GSK_Pilot_Project03032020/CT_RA_Tracker/ct_tt_ra"+str(date.today())+".xlsx")
    
    print("Shape of Dataframe1: "+str(df1.shape))
    print("Shape of Dataframe2: "+str(df2.shape))
     
     
    #Compare 2 dtaframes using Data Compy
    compare = datacompy.Compare(
                df1,
                df2,
                join_columns='Clinical Trial Identifier',  #You can also specify a list of columns
                #abs_tol=0, #Optional, defaults to 0
                #rel_tol=0, #Optional, defaults to 0
                #df1_name='Original', #Optional, defaults to 'df1'
                #df2_name='New' #Optional, defaults to 'df2'
                )
    ### compare.matches(ignore_extra_columns=False) = False ------> There is a change
    ### compare.matches(ignore_extra_columns=False) = True -------> There is no change
    ### Writing a logic to run when there is a change observed between two dataframes. ###
    if compare.matches(ignore_extra_columns=False) == True:    ##### To check changes in 2 dataframes ######
        print("There are no changes observed")
    
    else:
        print(compare.report())
        data = compare.intersect_rows ### compare.intersect_rows will give a dataframe with combination of 3 dataframes, they are df1, df2 and match, 
        data.to_csv('C:/Users/11201312/Documents/Techwave/GSK_Pilot_Project03032020/CT_RA_Tracker/comapre.intersect_rows_tt.csv') ### Store it in our location 
        #### Split data into 3 parts ##
                
        ####################### Part-1 - Dataframe1 #############################
        data1 = data[['clinical trial identifier', 'competitor asset_df1', 'sponsor(s)_df1', 'trial title_df1', 'condition/disease_df1', 
                'trial phase_df1', 'trial status_df1', 'other trial identifier(s)_df1', 'target accrual_df1', 'actual accrual_df1', 
                'title acronym_df1', 'countries_df1', 'trial objective(s)_df1', 'study design_df1', 'patient segments_df1', 'treatment plan_df1', 
                'dosing/comparator_df1', 'roa_df1', 'outcome measures_df1', 'study start date_df1', 'primary completion date_df1', 
                'study completion date_df1', 'inclusion criteria_df1', 'exclusion criteria_df1', 'trial tag/attribute_df1', 
                'other trial identifier(s)_df1', 'trial outcome(s)_df1']]
                
        ### Renaming Column names ###
        data1.columns = ['clinical trial identifier', 'competitor asset', 'sponsor(s)', 'trial title', 'condition/disease', 'trial phase',
                        'trial status', 'other trial identifier(s)', 'target accrual', 'actual accrual', 'title acronym', 'countries', 'trial objective(s)', 
                        'study design', 'patient segments', 'treatment plan', 'dosing/comparator', 'roa', 'outcome measures', 'study start date', 
                        'primary completion date', 'study completion date', 'inclusion criteria', 'exclusion criteria', 'trial tag/attribute', 
                        'other trial identifier(s)', 'trial outcome(s)']
                        
        ####################### Part-2 -  Dataframe2 ##############################
        data2 = data[['clinical trial identifier', 'competitor asset_df2', 'sponsor(s)_df2', 'trial title_df2', 'condition/disease_df2', 'trial phase_df2', 
                'trial status_df2', 'other trial identifier(s)_df2', 'target accrual_df2', 'actual accrual_df2', 'title acronym_df2', 'countries_df2', 
                'trial objective(s)_df2', 'study design_df2', 'patient segments_df2', 'treatment plan_df2', 'dosing/comparator_df2', 'roa_df2', 'outcome measures_df2', 
                'study start date_df2', 'primary completion date_df2', 'study completion date_df2', 'inclusion criteria_df2', 'exclusion criteria_df2',
                'trial tag/attribute_df2', 'other trial identifier(s)_df2', 'trial outcome(s)_df2']]
                
        ### Renaming Column names ###
        data2.columns = ['clinical trial identifier', 'competitor asset', 'sponsor(s)', 'trial title', 'condition/disease', 'trial phase',
                        'trial status', 'other trial identifier(s)', 'target accrual', 'actual accrual', 'title acronym', 'countries', 'trial objective(s)', 
                        'study design', 'patient segments', 'treatment plan', 'dosing/comparator', 'roa', 'outcome measures', 'study start date', 
                        'primary completion date', 'study completion date', 'inclusion criteria', 'exclusion criteria', 'trial tag/attribute', 
                        'other trial identifier(s)','trial outcome(s)']
        
        ######################### Part-3 - Match-Dataframe ###########################
        data_match = data[['clinical trial identifier', 'competitor asset_match', 'sponsor(s)_match', 'trial title_match', 'condition/disease_match', 
                    'trial phase_match', 'trial status_match', 'other trial identifier(s)_match', 'target accrual_match', 'actual accrual_match', 
                    'title acronym_match', 'countries_match', 'trial objective(s)_match', 'study design_match', 'patient segments_match', 
                    'treatment plan_match', 'dosing/comparator_match', 'roa_match', 'outcome measures_match', 'study start date_match',
                    'primary completion date_match', 'study completion date_match', 'inclusion criteria_match', 'exclusion criteria_match',
                    'trial tag/attribute_match', 'other trial identifier(s)_match', 'trial outcome(s)_match']]
                    
        ########## Finding row and column indications for Changes sheet ########
        k=data_match.shape[0]     ## No.of Rows
        l=data_match.shape[1]     ## No.of Columns
        r=[]                      ## list to store row no's
        c=[]                      ## list to store column no's
        column=[]                 ## list to store columns names
        NCT=[]                    ## list to store NCT no's
        cmptrast=[]               ## Competitor Asset
        condtn=[]                 ## Condition/Disease
        acrnm=[]                  ## Title Acronym
        #title=[]                  ## Trial Title  
        cntris=[]                 ## Countries     
        spnsr=[]                  ## Sponsor/Collaborator
        trlphs=[]                 ## Trial Phase
        status=[]                 ## Trial Status
        d1=[]                     ## list to store df1 elements
        d2=[]                     ## list to store df2 elements to know what is actual change
        for j in range(1,l,1):
            for i in range(0,k,1):
                if (data_match.iloc[i][j]==False):
                    print("column name:",data1.iloc[:,j].name)
                    column.append(data1.iloc[:,j].name)
                    r.append(i)
                    c.append(j)
                    NCT.append(data1.iloc[i,0])
                    cmptrast.append(data1.iloc[i,1])
                    condtn.append(data1.iloc[i, 4])
                    acrnm.append(data1.iloc[i, 10])
                    cntris.append(data1.iloc[i, 11])
                    spnsr.append(data1.iloc[i, 2])
                    trlphs.append(data1.iloc[i, 5])
                    status.append(data1.iloc[i, 6])            
                    d1.append(data1.iloc[i,j])
                    d2.append(data2.iloc[i,j])
                    print("row index is: ",i)            
                    print("column index is: ", j)
                    print("--------------------------")
                    
        ####################### Creating a dataframe to store changes ##############################
        Changes = pd.DataFrame()
        ## Storing lists as a column in a dataframe
        Changes['Clinical Trial Identifier'] = NCT
        Changes['Competitor Asset'] = cmptrast
        Changes['Condition/Disease'] = condtn
        Changes['Title Acronym'] = acrnm
        Changes['Countries'] = cntris
        Changes['Sponsor/Collaborator'] = spnsr
        Changes['Trial Phase'] = trlphs
        Changes['Trial Status'] = status
        Changes['Column_Changes'] = column
        Changes['Previous Value'] = d1
        Changes['Current Value'] = d2
        Changes['Date of update'] = date.today().strftime("%B %d, %Y")
        
        #### Shape of Changes ####
        print("Shape of Changes: "+str(Changes.shape))
        
        ##################### Import Column Tags/Measures #######################
        tag = pd.read_excel("C:/Users/11201312/Documents/Techwave/GSK_Pilot_Project03032020/CT_RA_Tracker/Trial Tag.xlsx")
        tag.head()
        
        ########### Merge tags column to Changes sheet #######################
        Changes = Changes.merge(tag, on='Column_Changes')
        Changes.columns
       
        ##### Sorting Changes ######
        Changes = Changes.sort_values(['Clinical Trial Identifier', 'Trial Tag'], ascending=[True, True])
        
        today_changes = Changes[['Clinical Trial Identifier', 'Competitor Asset', 'Condition/Disease', 'Title Acronym',  'Countries', 
                                 'Sponsor/Collaborator', 'Trial Phase', 'Trial Status', 'Column_Changes', 'Trial Tag', 'Previous Value',
                                 'Current Value', 'Date of update']]
        
        ##### Shape of Changes after adding Tag ####
        print("Shape of changes after adding tag: "+str(today_changes))
        
        
        #### Export Today changes to current directory ####
        today_changes.to_excel("C:/Users/11201312/Documents/Techwave/GSK_Pilot_Project03032020/CT_RA_Tracker/Changes"+str(date.today())+".xlsx", index=False)
        
        ## Calling Existing Changes history and storing in a dataframe ####
        changes_history = pd.read_excel("C:/Users/11201312/Documents/Techwave/GSK_Pilot_Project03032020/CT_RA_Tracker/Changes_History.xlsx")
        print("Shape of changes history: ",str(changes_history.shape))
        #changes_history.columns
        
        
        ########################### Appending Today Changes to existing changes history ###########################
        frames = [today_changes, changes_history]  ### Createing a frame to add changes 
        history = pd.concat(frames)  ### Concatenate all dataframes in frames
        print("history df shape: ", history.shape)
        
        ############### Exporting updated changes history with "Changes_History.xlsx" name to our directory for future use #####
        history.to_excel("C:/Users/11201312/Documents/Techwave/GSK_Pilot_Project03032020/CT_RA_Tracker/Changes_History.xlsx", index=False)

        ##### Replacing todays data with base sheet for tomorrow comparison. Run this line of code after completing regular comparision task ####
        data_comp.to_excel('C:/Users/11201312/Documents/Techwave/GSK_Pilot_Project03032020/CT_RA_Tracker/ct_tt_ra.xlsx', index=False)
        
if __name__ == "__main__":
    RA()

Shape of downloaded data:  (2462, 27)
Shape of NCT, Phase data Share by KC:  (42, 3)
Shape of vlookup (106, 57)
Shape of trialPatientSegments  (460, 4)
Shape of trialPatientSegments1  (106, 5)
Vlookup TT, PatientSegment data: (106, 58)
Shape of drugdata:  (116, 4)
Shape of drugdata1:  (106, 6)
Vlookup of TT PatientSegment & Drug data shape: (106, 59)
tt_ct shape:  (106, 15)
Shape of vlookup dataframe: (42, 17)
(42, 43)
Shape of latest CT dataframe:  (42, 35)
Shape of Comparision dataframe: (42, 27)
CT Data to display:  (42, 33)
Vlookup tpddata and Updated1: (41, 60)
TT data to dispaly:  (41, 33)
History df shape:  (83, 33)
Shape of Dataframe1: (42, 27)
Shape of Dataframe2: (42, 27)
There are no changes observed
